<a href="https://colab.research.google.com/github/RelaxingCoyote/LSTM_CNN_text_understanding/blob/main/LSTM_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерация комментариев с использованием LSTM

## Подключение библиотек

In [ ]:
import pandas as pd
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

Подключаем google-диск.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


## Загружаем датасет

In [ ]:
df_path = '/content/gdrive/MyDrive/Machine Learning/Нейронные сети/comments_categorized.csv'
df = pd.read_csv(df_path,lineterminator='\n', dtype={
    'videoId':'category',
    'emotions':'category',
    'category':'category',
})

df.head()

,videoId,id,clean_text,emoji,clean_tokens,lematized_tokens,emotions,emoji_group,emoji_sub_group,category
0,PBqGyMay8X4,UgwhEpkSostw9m8VdEF4AaABAg,"Ребят, это капец страшно СНИМАТЬ 2 ЧАСТЬ???","['😨', '😨', '😨', '😱']","['Ребят', 'это', 'капец', 'страшно', 'СНИМАТЬ'...","['ребята', 'это', 'капец', 'страшно', 'снимать...",neutral,"['Smileys & Emotion', 'Smileys & Emotion', 'Sm...","['face-concerned', 'face-concerned', 'face-con...",Entertainment
1,PBqGyMay8X4,UgzU9fqq7bEQ4SzEtVN4AaABAg,Муд Диана: только вылезла из истории с клоуном...,['🙂'],"['Муд', 'Диана', 'вылезла', 'истории', 'клоуно...","['муд', 'диана', 'вылезти', 'история', 'клоуно...",skip,['Smileys & Emotion'],['face-smiling'],Entertainment
2,PBqGyMay8X4,Ugyvt5GA39_531iQWqJ4AaABAg,Диана: Я НЕ БУДУ ПРОДОЛЖАТЬ!!! Так же Диана че...,[],"['Диана', 'БУДУ', 'ПРОДОЛЖАТЬ', 'Диана', 'неск...","['диана', 'быть', 'продолжать', 'диана', 'неск...",skip,[],[],Entertainment
3,PBqGyMay8X4,Ugy2458qWiizy-zgKd94AaABAg,Хотите завтра увидеть новую меня??? Я так изме...,['😥'],"['Хотите', 'завтра', 'увидеть', 'новую', 'изме...","['хотеть', 'завтра', 'увидеть', 'новый', 'изме...",negative,['Smileys & Emotion'],['face-concerned'],Entertainment
4,PBqGyMay8X4,UgwP2ITWxnxec_sa75t4AaABAg,"Диана: кричит в 3 часа ночи от страхаСоседи, к...",['🗿'],"['Диана', 'кричит', '3', 'часа', 'ночи', 'стра...","['диана', 'кричать', '3', 'час', 'ночь', 'стра...",skip,['Objects'],['other-object'],Entertainment


Уберём дубликаты и значения типа NAN

In [ ]:
df = df.dropna()
df = df.drop_duplicates()
df

,videoId,id,clean_text,emoji,clean_tokens,lematized_tokens,emotions,emoji_group,emoji_sub_group,category
0,PBqGyMay8X4,UgwhEpkSostw9m8VdEF4AaABAg,"Ребят, это капец страшно СНИМАТЬ 2 ЧАСТЬ???","['😨', '😨', '😨', '😱']","['Ребят', 'это', 'капец', 'страшно', 'СНИМАТЬ'...","['ребята', 'это', 'капец', 'страшно', 'снимать...",neutral,"['Smileys & Emotion', 'Smileys & Emotion', 'Sm...","['face-concerned', 'face-concerned', 'face-con...",Entertainment
1,PBqGyMay8X4,UgzU9fqq7bEQ4SzEtVN4AaABAg,Муд Диана: только вылезла из истории с клоуном...,['🙂'],"['Муд', 'Диана', 'вылезла', 'истории', 'клоуно...","['муд', 'диана', 'вылезти', 'история', 'клоуно...",skip,['Smileys & Emotion'],['face-smiling'],Entertainment
2,PBqGyMay8X4,Ugyvt5GA39_531iQWqJ4AaABAg,Диана: Я НЕ БУДУ ПРОДОЛЖАТЬ!!! Так же Диана че...,[],"['Диана', 'БУДУ', 'ПРОДОЛЖАТЬ', 'Диана', 'неск...","['диана', 'быть', 'продолжать', 'диана', 'неск...",skip,[],[],Entertainment
3,PBqGyMay8X4,Ugy2458qWiizy-zgKd94AaABAg,Хотите завтра увидеть новую меня??? Я так изме...,['😥'],"['Хотите', 'завтра', 'увидеть', 'новую', 'изме...","['хотеть', 'завтра', 'увидеть', 'новый', 'изме...",negative,['Smileys & Emotion'],['face-concerned'],Entertainment
4,PBqGyMay8X4,UgwP2ITWxnxec_sa75t4AaABAg,"Диана: кричит в 3 часа ночи от страхаСоседи, к...",['🗿'],"['Диана', 'кричит', '3', 'часа', 'ночи', 'стра...","['диана', 'кричать', '3', 'час', 'ночь', 'стра...",skip,['Objects'],['other-object'],Entertainment
...,...,...,...,...,...,...,...,...,...,...
1235386,XE-Evw8dQwA,UgwQixf8oF3dAvFwd_h4AaABAg,"как всегда ,топ молодец чувак",[],"['топ', 'молодец', 'чувак']","['топ', 'молодец', 'чувак']",speech,[],[],Gaming
1235387,XE-Evw8dQwA,UgxkOKWQIXy2IyrMXdh4AaABAg,Японцы вам харакири скоро подъедут,['😈'],"['Японцы', 'харакири', 'скоро', 'подъедут']","['японец', 'харакири', 'скоро', 'подъехать']",skip,['Smileys & Emotion'],['face-negative'],Gaming
1235388,XE-Evw8dQwA,UgyX_swfNNhIadpGFIt4AaABAg,С днём рождения!;-) желаю того чтоб ты не бол...,[],"['днём', 'рождения', 'желаю', 'болел', 'видюхи...","['день', 'рождение', 'желать', 'болеть', 'видю...",speech,[],[],Gaming
1235389,XE-Evw8dQwA,UgwWY9PC71HtVGrJx4d4AaABAg,Вот это ДА!!!,[],['это'],['это'],positive,[],[],Gaming



При прочтении csv-файла ячейки типа list имеют строковый вид. Приведём ячейки с токенами к виду типа list.

In [ ]:
df['clean_tokens'] = df['clean_tokens'].apply(eval)

## Подготовка данных для сети

Перед тем как отправить на вход сети текст комментариев, пронумеруем все уникальные символы и представим текстовые данные в виде последовательности чисел (индексов).

In [ ]:
# text_sample - 
def text_to_seq(text_sample):
    # расчёт частоты употребления отдельных символов
    char_counts = Counter(text_sample)
    # сортировка словоря частоты по убыванию
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    # вывод символов, отсортированных по частоте употребления
    print(sorted_chars)
    # словарь символов необходимо преобразовать в более удобный тип dict
    # словарь символ-индекс (type == dict)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    # словарь индекс-символ (type == dict)
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    # последовательность индексов
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    # последовательность чисел, словарь символ-индекс, словарь индекс-символ
    return sequence, char_to_idx, idx_to_char

Преобразуем текст комментариев в вектор.

In [ ]:
text_sample =  ' '.join(df['clean_text'])

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'а', 'е', 'т', 'и', 'н', 'с', 'р', 'л', 'к', 'в', 'м', 'д', 'у', 'п', 'я', 'ь', 'ы', 'б', 'ч', 'г', 'з', 'й', '!', '.', ',', 'ж', 'ш', ')', 'х', 'ю', 'А', '?', 'О', 'К', ':', 'э', 'С', 'Т', 'П', 'e', 'В', 'Н', '0', '-', 'ц', 'М', 'Е', 'И', 'a', 'o', 'ё', 'Я', 'ф', 'Д', '2', 'Р', '1', 'щ', 'i', 't', '"', 'Л', 's', '*', 'n', 'У', 'Б', 'r', 'l', 'h', 'Э', '(', 'Г', '3', 'd', 'u', 'З', 'Ч', 'm', 'Х', '5', 'c', '❤', '4', 'y', 'g', 'D', 'Ж', 'p', '9', 'k', 'w', '6', 'Ы', 'f', 'Й', 'Ь', 'Ф', '8', 'b', '7', 'v', 'Ш', 'Ю', '_', 'S', 'A', 'I', 'T', 'O', 'M', 'E', 'P', 'R', "'", 'L', 'C', ';', 'x', 'G', 'N', 'W', 'H', '/', 'B', '\r', 'F', '=', '+', '♥', 'Ц', 'z', 'ъ', 'X', 'ا', '^', 'Y', 'Ё', 'і', 'U', 'Щ', 'j', '«', '»', 'V', 'K', 'ل', '%', '☺', '—', 'ه', 'ي', '@', '\xa0', '>', '<', '✌', '~', 'م', '♡', 'و', '✨', 'J', 'ن', '•', 'ر', 'ع', '°', 'q', '♀', 'ب', '|', '–', '$', '…', 'ت', '\\', 'ك', 'є', ']', '♂', '\u3000', '’', '✊', 'ë', '！', 'Z', '͡', '[', 'ح', '©', '☝', 'د', '”', 'ف', '☀',

## Генератор батчей из комментариев

In [ ]:
# Длинна последовательности в символах
SEQ_LEN = 256
# размер батча
BATCH_SIZE = 16

def get_batch(sequence):
    # последовательность для обучения
    trains = []
    # контрольная последовательность
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        # последовательности контрольная и тренировочная
        # отличаются первым и последним символом
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        # формируем батч
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

## Функция, генерирующаая текст

In [ ]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

## Класс нейросети

In [ ]:
# класс для RNN
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3c52dfe5-3625-4942-9326-3a74c50ec1a0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

## Создаем нейросеть и обучаем ее

In [ ]:
# инициализируем device и модель
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

# метрика, алгоритм оптимизации, ?
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

# эпохи
n_epochs = 50000
# средняя ошибка
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    # рассчитаем ошибку
    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    # градиент
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    # каждые 50 итераций выводим среднюю ошибку
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 3.9264296245574952
 н          во  че а          о  ви  е     и     о  с         о     к е ло     а  и   и  пи   в   н    и  и     то     ро о            то  л        ва       е ве   о но сока  т л     о о е   тпи  но в 
Loss: 3.3761859130859375
 тисво но си са баса     со но е срио дива  песно на ло но   со са вона то тосто тасе то кос со не со покова до сво ко а не и но по  не се не су то ва па о ко по вет ны по  со со пат та сешо со по вома
Loss: 3.2307371377944945
 поли ко не и то по то до поте пото ко со ко косо то роло то са  то А те лол то перако  но то ло до но с ко во  о во мо то ра  па роте сос мо ла поро паси но ви но но но то до по поле то не не поте тя 
Loss: 3.1584607553482056
 и но полотро ваго тро на попота восо то тол то  те дот пал ват тол лала сас пола тот то маму то мом ла по вама то кова на со то ва тат  вава ложи вомта ви вива та сево мона то довить на вит кода вама 
Loss: 3.13942440032959
 ва толей вевика на мак том толь подолемоди с  вот докот не поводо  д

## Генерируем текст

In [ ]:
model.eval()
print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='. '
    )
)

.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             Стала подп

## Свёрточная нейронная сеть

In [ ]:
class CNN(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d()
      self.conv2 = nn.Conv2d()

      self._non_linear = None

      x = torch.randn()

      x = self.convs(x)

      self.fcn1 = nn.Linear()
      self.fcn2 = nn.Linear()
      self.fcn3 = nn.Linear()


    def convs(self,x):
      pass

## Над чем стоит поработать?

Добавить:

*   Работу с датафремом, а не со сплошным текстом
*   Разметку данных
*   Скармливать текст комментариев по категориям
*   Как задать тему для генерации текста?
